## Modified Great Hammerhead Von Bertalanffy Age Proxy with a Vertebral-Aged Correction Coefficient


Great Hammerhead (GHH) sharks are long lived, large, semi-pelagic elasmobranchs. When studied and sampled for research purposes, morphometric measurements are taken, usually including lengths, mass, sex, and chemical (stable isotope or toxicant) analysis. 

Relationships between ontogeny and toxicants are vital for toxilogical studies in large fauna. While lengths can be easily documented, aging large elasmobranchs proves to be more of an estimation, and is difficult to do without proper instrumentation, and require vertebrae sections with bomb radiocarbon analysis for validation.

In the current study, lengths were taken, but vertebral sections were not extracted. In order to examine relationships between toxicants throughout ontogeny, age must be proxied.

This notebook provides a method for aging GHH utilizing mathamatical models with a correction coefficient derived from previously recorded vertebral measurements. Code provided assumes that your local dataset contains the following morphometric values: Total Length (cm), Fork Length (cm), Sex (Male, Female)

In [ ]:
# Set up your packages, datasets for the pipeline

# Import All Packages
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

# Set working directory
os.chdir('/Users/razzywachtel/Desktop/GHH') # set your own directory for your dataset 

# Read csv with pandas
Adult_specimens = pd.read_csv('GHH_CSV.csv') # use your local dataset, make sure lengths are in cm

### Impute Missing Fork Length Values 

We use Fork Length (FL (cm)) to calculate Von Bertalanffy Age. If samples were not measured with FL, we must convert Total Length (TL (cm)) to FL using a predefined relationship (specific to Great Hammerhead Sharks) (Piercy et al 2010):

TL = 1.2533(FL) + 3.472

In [ ]:
# If the only measured length is Total Length (TL), convert to Fork Length (FL)
if 'FL (mm)' not in Adult_specimens.columns:
    Adult_specimens['FL (mm)'] = (Adult_specimens['TL (mm)'] - 3.472)/ 1.2533

print(Adult_specimens)

### Define Von Bertalanffy Function For GHH 

Now that all data is successfully imputed to the csv, we proceed by defining a modified Von Bertalanffy (VB) function (Piercy et al 2010):

### Lt = L∞(1 – be^(-kt))

where male and female asymptotic lengths (L∞) were set at 264.2 and 323.9 cm, respectively, growth coefficient (k) was 0.16 (male) and 0.11 (female), and fork length at birth (b) was set at 50 cm for both sexes following previous works regarding Northwest Atlantic GHH (Piercy et al. 2014; Driggers et al. 2021). 

In [12]:
# Define the function to calculate age for males
def calculate_male_age(Lt, L8=264.2, b=50, k=0.16):
    with np.errstate(divide='ignore', invalid='ignore'): # remove np Runtime Warnings
        Age = np.log((1/b) * (1 - (Lt / L8))) / (-k) # VB equation
        if Age <= 0 or np.isinf(Age) or np.isnan(Age): # when FL exceeds hypothetical asymptotic age, it results in a calculation error. 
            return np.nan  # remove any invalid calculations
        else:
            return Age

# Define the function to calculate age for females
def calculate_female_age(Lt, L8=323.9, b=50, k=0.11):
    with np.errstate(divide='ignore', invalid='ignore'): #remove np Runtime Warnings
        Age = np.log((1/b) * (1 - (Lt / L8))) / (-k) # VB equation
        if Age <= 0 or np.isinf(Age) or np.isnan(Age): # when FL exceeds hypothetical asymptotic age, it results in a calculation error. 
            return np.nan  # remove any invalid calculations
        else:
            return Age

# Define the wrapper function to apply the correct calculation based on 'Sex'
def calculate_age(row):
    if row['Sex'] == 'Male':
        return calculate_male_age(row['FL (mm)'])
    elif row['Sex'] == 'Female':
        return calculate_female_age(row['FL (mm)'])
    else:
        return np.nan  

# Assuming Adult_Specimens is your DataFrame and has columns 'Sex' and 'FL (mm)'
# Apply the function to create a new 'Proxied Age' column
Adult_specimens['Modified VB Age'] = Adult_specimens.apply(calculate_age, axis=1)

# Find summary statistics of the new 'Proxied Age' Column 
Adult_specimens.describe()


,TL (mm),FL (mm),Estimated Age (Years),"Muscle Mercury (THg, ppm ww)",Modified VB Age,Age Corrected Proxy
count,41.000000,41.000000,34.000000,41.000000,0.0,0.0
mean,2812.376829,2120.579488,43.618176,3.523067,NaN,NaN
std,803.568110,718.846934,6.334734,1.525758,NaN,NaN
min,1193.000000,293.089000,30.319000,0.869000,NaN,NaN
25%,2112.000000,1578.000000,41.094750,2.293800,NaN,NaN
50%,2740.000000,2046.820000,42.435000,3.660700,NaN,NaN
75%,3486.000000,2610.000000,47.845250,4.233200,NaN,NaN
max,4390.000000,3571.000000,60.479000,9.151400,NaN,NaN


### Ground Von Bertalanffy Equation With Vertebral Aged GHH

Unfortunately, for the set of Great Hammerheads from the Northwest Atlantic Ocean in this study, the VB age estimation grossly overestimated ages. In order to provide better accuracy, we must add an age correction coefficient to the values calculated from the function above.

The most accurate aging technique for sharks is vertebral-based aging. The Florida Wildlife Commission, Fish and Wildlife Research Institute (FWC - FWRI), were able to provide two female Great Hammerhead subjects for vertebral aging comparisons. If you have additional vertebral aged Northwest Atlantic Great Hammerhead sharks, add these to the list below, and use their respective age calculation (by sex) in the following cells.

FWC - FWRI recorded values (FL (cm), Vertebral Age (yrs)): 
1) (292, 16.5)
2) (243, 11.5)



In [ ]:
Vertebral_aged_FL = [292, 243] # list of vertebral aged lengths in cm 

Vertebral_vb_estimation = [calculate_female_age(Lt) for Lt in Vertebral_aged_FL] # list of VB ages for vertebral aged females

print(Vertebral_vb_estimation) # this list will contain 

We now create a regression line for these two x - y coordinates (Vertebral age, VB age)

In [ ]:
Vertebral_vb_array = np.array([[16.5, 56.635017], [11.5, 48.174946]]) # create an array of the Vertebral vs VB ages
Vertebral_vb_df = pd.DataFrame(Vertebral_vb_array) # transform to pd Dataframe
Vertebral_vb_df.columns = ['Vertebral Age', 'VB Age'] # name columns 

Vb_age_array = Vertebral_vb_df['VB Age'].values.reshape(-1, 1) # reshape to two dimensions to properly fit regression model
Vertebral_age_array = Vertebral_vb_df['Vertebral Age'].values.reshape(-1, 1)  # reshape to two dimensions to properly fit regression model

# Set up linear regression 
model = LinearRegression()

model.fit(Vb_age_array, Vertebral_age_array) # fit arguments to regression

slope = model.coef_[0][0] # extract slope from model
intercept = model.intercept_[0] # extract intercept

# Predicting y-values based on the model
y_pred = model.predict(Vertebral_age_array) # find y predicted values

print(f"Slope: {slope}, Intercept: {intercept}")

Now that we have our regression line, we now place our predefined VB ages along this line as a correction coefficient.

In [ ]:
def correction_coefficient(m):
    Age_correction = m * 0.5910115884370238 - 16.971951358327846 # regression slope and intercept -- use your own regression values in this line
    return Age_correction

# Apply the function to create a new 'Age Corrected Proxy' column
Adult_specimens['Age Corrected Proxy'] = Adult_specimens['Modified VB Age'].apply(correction_coefficient)

print(Adult_specimens) # check proxied ages with the applied correction coefficient

In [ ]:
Adult_specimens.to_csv('Age Corrected Von Bertalanffy Great Hammerheads', index = False) # export df as csv locally

Citations:

Piercy AN, Carlson JK, Passerotti MS. 2010. Age and growth of the great hammerhead shark, Sphyrna mokarran, in the north-western Atlantic Ocean and Gulf of Mexico. Mar Freshwater Res. 61(9):992. doi:10.1071/MF09227.